<a href="https://colab.research.google.com/github/raishagun/Election-Prediction/blob/master/Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/raishagun/Election-Prediction.git

fatal: destination path 'Election-Prediction' already exists and is not an empty directory.


In [26]:
import tweepy 
import pandas as pd
from tweepy import OAuthHandler

consumer_key= "DLhoHkK6uOGz3Ni9ybuoruiLm"
consumer_secret  = "jKyrDR68msnbmgZMNNJdSJwadGoed11imKV1jdlNYNczL9HvEk"
access_token ="1289037463684489217-5PddOsmU9h5deRwOhdigvhg32FEgID"
access_token_secret= "cAPevFduWkiNP3juu8WGAwZm69vgv8C4s8RPz0WE8MJFK"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
# authentication of access token and secret 
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth,wait_on_rate_limit = True)


date1 = "2020-10-28"
date2 = "2020-11-09"


text = []


def get_tweets(date1,date2,word):
    count = 0
    for tweet in tweepy.Cursor(api.search , q=word,count =1000,lang="en",since_id = date1,until = date2).items():
        print(tweet.text)
        text.append(tweet.text)
        print('<--------------------------------------------------->')
        count+=1
        print(count)
    
get_tweets(date1,date2,"#amitshah bjp")  
           
data1 = list(zip(text))
df1 = pd.DataFrame(data =data1, columns =["Tweet"])

df1.to_csv('bjp11.csv') 


Streaming output truncated to the last 5000 lines.
<--------------------------------------------------->
433
RT @MJALSHRIKA: This is the Arabs' direct message to the leaders of the terrorist Hindutva outfits RSS,BJP, VHP, Bajrang Dal and ABVP, eith…
<--------------------------------------------------->
434
RT @MJALSHRIKA: This is the Arabs' direct message to the leaders of the terrorist Hindutva outfits RSS,BJP, VHP, Bajrang Dal and ABVP, eith…
<--------------------------------------------------->
435
#AmitShah it’s a shame what Arnab Goswami has to go through. That too when BJP in centre. Imagine what wud happen i… https://t.co/aE5Iwx9IDB
<--------------------------------------------------->
436
RT @MJALSHRIKA: This is the Arabs' direct message to the leaders of the terrorist Hindutva outfits RSS,BJP, VHP, Bajrang Dal and ABVP, eith…
<--------------------------------------------------->
437
RT @MJALSHRIKA: This is the Arabs' direct message to the leaders of the terrorist Hindutva outf

# **Importing Libraries**

In [23]:
import re
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd

import datetime as dt
from collections import Counter

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import words
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# **Cleaning** **BJPDataset1**

```
# length of the dataset11
```



In [28]:
tweets_list = df1['Tweet'].tolist()
len(tweets_list)

1783

# Removing duplicate tweets if any

In [29]:
tweets_list_updated = set(tweets_list)
len(tweets_list_updated)

731

```
No of duplicate tweets
```

In [30]:
df1['Tweet'].duplicated().sum()

1052

In [31]:
df1 = df1.drop_duplicates(subset = 'Tweet', keep = 'first',ignore_index = True)
df1.shape

(731, 1)

`Removed all the dupliactes`



# **Removing Emojis**


In [32]:
def remove_emoji(tweet):
    emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', tweet)

df1['Processed_Tweet']=df1.Tweet.apply(remove_emoji)

From now on, all the preprocessing & modelling on tweets will be stored in the data column called 'Processed_Tweet'

In [35]:
count = 0
for i in df1['Processed_Tweet']:
    if re.match(r'^RT', i):  #remove any rows that contain RT
        df1.drop(df1.loc[df1['Processed_Tweet']==i].index,inplace = True)
        count+=1
    
count

147

In [36]:
df1.shape

(584, 2)

In [37]:
df1['Processed_Tweet']

1      We need leadership change immediately #pmofind...
2      #thooktahaibharat on BJP, #NarendraModi ,#Amit...
3      Beat it so much that #ArnabGoswamy should neve...
4      Extremely sad to see the condition of this cou...
5      #NarendraModi #AmitShah #YogiAdityanath #BJP M...
                             ...                        
726    Union Home Minister #AmitShah to hold a 45 min...
727    WEST BENGAL \nUnion Home Minister Amit Shah to...
728    @narendramodi Pls public  ko support karo baad...
729    #AmitShah #NarendraModi .\n\nDear Sir\nAs per ...
730    Wake up #BJP #Karyakartas!! If #AmitShah doesn...
Name: Processed_Tweet, Length: 584, dtype: object

# **Removing Username**

In [38]:
df1['Processed_Tweet'] = [re.sub('@[^\s]+', ' ', i) for i in df1['Processed_Tweet']] #remove usernames
df1['Processed_Tweet']

1      We need leadership change immediately #pmofind...
2      #thooktahaibharat on BJP, #NarendraModi ,#Amit...
3      Beat it so much that #ArnabGoswamy should neve...
4      Extremely sad to see the condition of this cou...
5      #NarendraModi #AmitShah #YogiAdityanath #BJP M...
                             ...                        
726    Union Home Minister #AmitShah to hold a 45 min...
727    WEST BENGAL \nUnion Home Minister Amit Shah to...
728      Pls public  ko support karo baad vote mango ...
729    #AmitShah #NarendraModi .\n\nDear Sir\nAs per ...
730    Wake up #BJP #Karyakartas!! If #AmitShah doesn...
Name: Processed_Tweet, Length: 584, dtype: object

# **Removing URL if any**

In [39]:
df1['Processed_Tweet'] = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', i) for i in df1['Processed_Tweet']] #Remove urls
df1['Processed_Tweet']

1      We need leadership change immediately #pmofind...
2      #thooktahaibharat on BJP, #NarendraModi ,#Amit...
3      Beat it so much that #ArnabGoswamy should neve...
4      Extremely sad to see the condition of this cou...
5      #NarendraModi #AmitShah #YogiAdityanath #BJP M...
                             ...                        
726    Union Home Minister #AmitShah to hold a 45 min...
727    WEST BENGAL \nUnion Home Minister Amit Shah to...
728      Pls public  ko support karo baad vote mango ...
729    #AmitShah #NarendraModi .\n\nDear Sir\nAs per ...
730    Wake up #BJP #Karyakartas!! If #AmitShah doesn...
Name: Processed_Tweet, Length: 584, dtype: object

# **Removing Hashtags but keeping the word**

In [40]:
df1['Processed_Tweet'] = [re.sub(r'#([^\s]+)', r'\1', i) for i in df1['Processed_Tweet']] # remove the # in #hashtag
df1['Processed_Tweet']

1      We need leadership change immediately pmofindi...
2      thooktahaibharat on BJP, NarendraModi ,AmitSha...
3      Beat it so much that ArnabGoswamy should never...
4      Extremely sad to see the condition of this cou...
5      NarendraModi AmitShah YogiAdityanath BJP My  r...
                             ...                        
726    Union Home Minister AmitShah to hold a 45 minu...
727    WEST BENGAL \nUnion Home Minister Amit Shah to...
728      Pls public  ko support karo baad vote mango ...
729    AmitShah NarendraModi .\n\nDear Sir\nAs per re...
730    Wake up BJP Karyakartas!! If AmitShah doesn't ...
Name: Processed_Tweet, Length: 584, dtype: object

# **Using Preprocessor Library**

In [46]:
!pip install preprocessor

In [48]:
!pip install tweet-preprocessor

In [49]:
import preprocessor as p #Module for faster preprocessing specifically for tweets
df1['Processed_Tweet'] = [p.clean(i) for i in df1['Processed_Tweet']]
df1['Processed_Tweet']

AttributeError: ignored